In [10]:
#-*- coding:utf-8 -*-
from func import *
import os
import pandas as pd
import json
import numpy as np
from PIL import Image
import cv2
import metric
import pprint
import argparse
import pickle
from sklearn.metrics import accuracy_score,roc_curve,auc
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from openpyxl import load_workbook
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 10


UINT8_MAX = np.iinfo(np.uint8).max
UINT16_MAX = np.iinfo(np.uint16).max

#from visual import make_overlay_image, make_overlay_image2
#import visual as visual_lib

In [2]:
def convert_dict_to_array(contour):
    arr = [[row["x"], row["y"]] for row in contour]
    arr = np.array(arr)
    return arr

In [3]:
def get_masks(contours, width, height, dtype=np.float32, mask_size=1000):
    assert isinstance(contours, list) or isinstance(contours, tuple)

    images = []
    for contour in contours:
        for conts in contour:
            images.append(get_mask(conts, width, height, dtype, mask_size=mask_size))

    return images

In [4]:
def get_mask(contour, width, height, dtype=np.float32, mask_size=1000):
    if width < height:
        mask_height = mask_size
        mask_width = int(mask_size * width / height)
    else:
        mask_width = mask_size
        mask_height = int(mask_size * height / width)

    image = np.zeros((mask_height, mask_width), np.uint8)

    assert len(contour) > 0

    for subcontour in contour:
        assert len(subcontour) > 0
        pts = np.array(
            [[[(x + width / 2) / width * mask_width, (y + height / 2) / height * mask_height]] for x, y in subcontour],
            np.int32)
        cv2.fillPoly(image, [pts], (255, 255, 255))
    return convert_image_type(image, dtype)

In [5]:
def convert_image_type(image, dtype=np.float32):
    if image.dtype == np.uint8:

        if dtype == np.float32:
            image = image.astype(np.float32)
            image /= UINT8_MAX
            return image
        elif dtype == np.uint8:
            return image
        else:
            raise TypeError('numpy.float32 or numpy.uint8 supported as a target dtype')

    elif image.dtype == np.uint16:

        if dtype == np.float32:
            image = image.astype(np.float32)
            image /= UINT16_MAX
            return image
        elif dtype == np.uint8:
            image = image.astype(np.float32)
            image *= UINT8_MAX / UINT16_MAX
            image = image.astype(np.uint8)
            return image
        elif dtype == np.uint16:
            return image
        else:
            raise TypeError('numpy.float32 or numpy.uint8 or numpy.uint16 supported as a target dtype')

    else:
        raise TypeError('numpy.uint8 or numpy.uint16 supported as an input dtype')

In [6]:
def get_human_output(pixel_array, human_data):
    _dict = eval(human_data['contour_list'])
    _ratings = eval(human_data['rating_list'])

    height, width = pixel_array.shape

    if _ratings:
        masks = []
        for _key in _dict.keys():
            _contour = _dict[_key]

            for _rating in _ratings:
                if _rating['contourId']  == _key:
                    lesion_rating = _rating['rating']

            arr = convert_dict_to_array(_contour)

            arr[..., :, 0] = arr[:, 0] * width - width / 2
            arr[..., :, 1] = arr[:, 1] * height - height / 2

            arr = arr.astype(np.int64)

            arr = np.expand_dims(arr, 0)
            arr = np.expand_dims(arr, 0)

            mask = sum(get_masks([arr], width=width, height=height, mask_size=max(width, height)))
            mask = mask * (float(lesion_rating)/5)
            masks.append(mask)

        final_mask = np.stack(masks, axis=0).max(0)

    else:
        final_mask = np.zeros((height, width), dtype=np.float32)

    return final_mask

In [7]:
def get_ai_output(human_json_full):
    masks = []
    prob_map, prob = json_ai_map_open(human_json_full)
    np_map = np.array(prob_map)
    cutoff_mask = cutoff(np_map,0.15)
    masks.append(cutoff_mask)
    final_mask = np.stack(masks, axis=0).max(0)
    
    return final_mask

In [8]:
def get_gt_final_mask(data):
    if 'abnormal_finding' in data.keys() and data['abnormal_finding']:
        masks = []
        for _dict in data['abnormal_finding']:
#             if _dict['label_text'] in mca_list:
            if _dict['label_text']:
                for contour_key in _dict['contour_list'].keys():
                    contour = _dict['contour_list'][contour_key]
                    arr = convert_dict_to_array(contour)
                    # print(width, height)
                    # print(arr)

                    # print(width, height)
                    # print(arr)

                    arr[..., :, 0] = arr[:, 0]
                    arr[..., :, 1] = arr[:, 1]
                    arr = arr.astype(np.int64)

                    arr = np.expand_dims(arr, 0)
                    arr = np.expand_dims(arr, 0)

                    mask = sum(get_masks([arr], width=width, height=height, mask_size=max(width, height)))
                    masks.append(mask)
            else:
                mask = np.zeros((height, width), dtype=np.float32)
                masks.append(mask)

        final_mask = np.stack(masks, axis=0).max(0)
    else:
        final_mask = np.zeros((height, width), dtype=np.float32)
        
    return final_mask

In [16]:
df = pd.read_excel('./1015_updated.xlsx')

In [26]:
name_li = df['allocation']
visible_li = df['GT_visible']
clear_visible_li = df['GT_clear_visible']

In [37]:
visible = df['GT_visible']==1 
hospital = df['Inst']=='K'
visible_patient = df[visible&hospital]['allocation']

In [46]:
visible_patient = visible_patient.tolist()
len(visible_patient)

161

In [ ]:
gt_masks = []
human_masks = []
# hospital_name = 'brmh'
hospital_name = 'kyuh'
# hospital_name = 'gugh'

visible = df['GT_visible']==1 
hospital = df['Inst']=='K'
visible_patient = df[visible&hospital]['allocation']
visible_patient_li = visible_patient.tolist()


json_root_path = './data/review_result_20200705/{}-A2/{}/healthcheck'.format(hospital_name.upper(), hospital_name) # ['K-A2-0001.dcm.json', 'K-A2-0002.dcm.json', 'K-A2-0003.dcm.json', 'K-A2-0004.dcm.json', 'K-A2-0005.dcm.json']
# file_name_li  = os.listdir(json_root_path)

for file_name in visible_patient_li:
    file_name = file_name+'.dcm.json'
    json_file = os.path.join(json_root_path, (file_name))

    with open(json_file, "r") as f:
        data = json.load(f)
        
    height, width = data['height'], data['width']
    pixel_array = np.zeros((height,width))
    
    gt_masks.append(get_gt_final_mask(data))

human_root_path = './data/ai_json(all files)/{}_healthcheck/json_output'.format(hospital_name) # ['K-A2-0001.json','K-A2-0002.json','K-A2-0003.json', 'K-A2-0004.json']
# human_json_name_li = os.listdir(human_root_path)

for human_json_name in visible_patient_li:
    human_json_name=human_json_name+'.json'
    human_json_full = os.path.join(human_root_path, human_json_name)
    human_masks.append(get_ai_output(human_json_full))

new_shape = (512, 512)
human_outputs = [cv2.resize(np.asarray(human_mask), new_shape, interpolation=cv2.INTER_NEAREST) for human_mask in human_masks]

gt_masks = [gt_mask.astype(bool) for gt_mask in gt_masks]


resized_human_outputs = []
for index, human_output in enumerate(human_outputs):
    resized_human_outputs.append(np.resize(human_output, gt_masks[index].shape))

jafroc_value = metric.jafroc(resized_human_outputs, gt_masks)
print("jafroc:\t{:.3f}".format(jafroc_value))

jaf_ci = metric.bootstrap_jafroc_ci(resized_human_outputs, gt_masks, n_bootstraps=100, alpha=0.05, rng_seed=123)
print(jaf_ci)
              
# csvfile.close()

In [10]:
gt_masks = []
human_masks = []
hospital_name = 'brmh'
# hospital_name = 'kyuh'
# hospital_name = 'gugh'

json_root_path = './data/review_result_20200705/{}-A2/{}/healthcheck'.format(hospital_name.upper(), hospital_name)
file_name_li  = os.listdir(json_root_path)

for file_name in file_name_li:
    json_file = os.path.join(json_root_path, (file_name))

    with open(json_file, "r") as f:
        data = json.load(f)
        
    height, width = data['height'], data['width']
    pixel_array = np.zeros((height,width))

    gt_masks.append(get_gt_final_mask(data))

human_root_path = './data/ai_json(all files)/{}_healthcheck/json_output'.format(hospital_name)
human_json_name_li = os.listdir(human_root_path)

for human_json_name in human_json_name_li:
    human_json_full = os.path.join(human_root_path, human_json_name)
    human_masks.append(get_ai_output(human_json_full))

new_shape = (512, 512)
human_outputs = [cv2.resize(np.asarray(human_mask), new_shape, interpolation=cv2.INTER_NEAREST) for human_mask in human_masks]

gt_masks = [gt_mask.astype(bool) for gt_mask in gt_masks]


resized_human_outputs = []
for index, human_output in enumerate(human_outputs):
    resized_human_outputs.append(np.resize(human_output, gt_masks[index].shape))

jafroc_value = metric.jafroc(resized_human_outputs, gt_masks)
print("jafroc:\t{:.3f}".format(jafroc_value))

jaf_ci = metric.bootstrap_jafroc_ci(resized_human_outputs, gt_masks, n_bootstraps=100, alpha=0.05, rng_seed=123)
print(jaf_ci)
              
# csvfile.close()

jafroc:	0.667
(0.631, 0.707)


In [11]:
gt_masks = []
human_masks = []

# hospital_name = 'brmh'
# hospital_name = 'kyuh'
hospital_name = 'gugh'

json_root_path = './data/review_result_20200705/{}-A2/{}/healthcheck'.format(hospital_name.upper(), hospital_name)
file_name_li  = os.listdir(json_root_path)

for file_name in file_name_li:
    json_file = os.path.join(json_root_path, (file_name))

    with open(json_file, "r") as f:
        data = json.load(f)
        
    height, width = data['height'], data['width']
    pixel_array = np.zeros((height,width))

    gt_masks.append(get_gt_final_mask(data))

human_root_path = './data/ai_json(all files)/{}_healthcheck/json_output'.format(hospital_name)
human_json_name_li = os.listdir(human_root_path)

for human_json_name in human_json_name_li:
    human_json_full = os.path.join(human_root_path, human_json_name)
    human_masks.append(get_ai_output(human_json_full))

new_shape = (512, 512)
human_outputs = [cv2.resize(np.asarray(human_mask), new_shape, interpolation=cv2.INTER_NEAREST) for human_mask in human_masks]

gt_masks = [gt_mask.astype(bool) for gt_mask in gt_masks]


resized_human_outputs = []
for index, human_output in enumerate(human_outputs):
    resized_human_outputs.append(np.resize(human_output, gt_masks[index].shape))

jafroc_value = metric.jafroc(resized_human_outputs, gt_masks)
print("jafroc:\t{:.3f}".format(jafroc_value))

jaf_ci = metric.bootstrap_jafroc_ci(resized_human_outputs, gt_masks, n_bootstraps=100, alpha=0.05, rng_seed=123)
print(jaf_ci)
              
# csvfile.close()

jafroc:	0.561
(0.531, 0.582)


In [12]:
gt_masks = []
human_masks = []

# hospital_name = 'brmh'
# hospital_name = 'kyuh'
# hospital_name = 'gugh'

hospital_names = ['brmh', 'kyuh', 'gugh']
for hospital_name in hospital_names:
    json_root_path = './data/review_result_20200705/{}-A2/{}/healthcheck'.format(hospital_name.upper(), hospital_name)
    file_name_li  = os.listdir(json_root_path)

    for file_name in file_name_li:
        json_file = os.path.join(json_root_path, (file_name))

        with open(json_file, "r") as f:
            data = json.load(f)

        height, width = data['height'], data['width']
        pixel_array = np.zeros((height,width))

        gt_masks.append(get_gt_final_mask(data))

    human_root_path = './data/ai_json(all files)/{}_healthcheck/json_output'.format(hospital_name)
    human_json_name_li = os.listdir(human_root_path)

    for human_json_name in human_json_name_li:
        human_json_full = os.path.join(human_root_path, human_json_name)
        human_masks.append(get_ai_output(human_json_full))

new_shape = (512, 512)
human_outputs = [cv2.resize(np.asarray(human_mask), new_shape, interpolation=cv2.INTER_NEAREST) for human_mask in human_masks]

gt_masks = [gt_mask.astype(bool) for gt_mask in gt_masks]


resized_human_outputs = []
for index, human_output in enumerate(human_outputs):
    resized_human_outputs.append(np.resize(human_output, gt_masks[index].shape))

jafroc_value = metric.jafroc(resized_human_outputs, gt_masks)
print("jafroc:\t{:.3f}".format(jafroc_value))

jaf_ci = metric.bootstrap_jafroc_ci(resized_human_outputs, gt_masks, n_bootstraps=100, alpha=0.05, rng_seed=123)
print(jaf_ci)
              
# csvfile.close()

jafroc:	0.652
(0.637, 0.67)


In [17]:
gt_masks = []
human_masks = []

# hospital_name = 'brmh'
# hospital_name = 'kyuh'
# hospital_name = 'gugh'

# hospital_names = ['brmh', 'kyuh', 'gugh']
# for hospital_name in hospital_names:
json_root_path = './data/gt_json(opt)/healthcheck'
file_name_li  = os.listdir(json_root_path)

for file_name in file_name_li:
    json_file = os.path.join(json_root_path, (file_name))

    with open(json_file, "r") as f:
        data = json.load(f)

    height, width = data['height'], data['width']
    pixel_array = np.zeros((height,width))

    gt_masks.append(get_gt_final_mask(data))

human_root_path = './data/ai_json(opt)/healthcheck/json_output'
human_json_name_li = os.listdir(human_root_path)

for human_json_name in human_json_name_li:
    human_json_full = os.path.join(human_root_path, human_json_name)
    human_masks.append(get_ai_output(human_json_full))

new_shape = (512, 512)
human_outputs = [cv2.resize(np.asarray(human_mask), new_shape, interpolation=cv2.INTER_NEAREST) for human_mask in human_masks]

gt_masks = [gt_mask.astype(bool) for gt_mask in gt_masks]


resized_human_outputs = []
for index, human_output in enumerate(human_outputs):
    resized_human_outputs.append(np.resize(human_output, gt_masks[index].shape))

jafroc_value = metric.jafroc(resized_human_outputs, gt_masks)
print("jafroc:\t{:.3f}".format(jafroc_value))

jaf_ci = metric.bootstrap_jafroc_ci(resized_human_outputs, gt_masks, n_bootstraps=100, alpha=0.05, rng_seed=123)
print(jaf_ci)

# csvfile.close()

jafroc:	0.679
(0.595, 0.749)
